<h1><center> Importing Libraries </h1></center>


In [1]:
import pandas as pd
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

 <h1><center>Indices and Equities of Choice</h1></center>

<h2>Indices</h2>
<ol>
    <li> S&P 500 </li>
    <li> NASDAQ </li>
    <li> FTSE 100 </li>
    <li> STI Index </li>
    <li> KOSPI Composite Index </li>
    </ol>

<h2>Equities</h2>
<ol>
    <li> Amazon </li>
    <li> Apple </li>
    <li> Google </li>
    <li> Microsoft </li>
    <li> Netflix </li>
</ol>

<h1><center> Downloading the Data </h1></center>

In [2]:
msft_data = yf.download("msft", start="2010-01-01", end="2023-05-01")
aapl_data = yf.download("aapl", start="2010-01-01", end="2023-05-01")
goog_data = yf.download("goog", start="2010-01-01", end="2023-05-01")
amzn_data = yf.download("amzn", start="2010-01-01", end="2023-05-01")
nflx_data = yf.download("nflx", start="2010-01-01", end="2023-05-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
sp500_data = yf.download("^gspc", start="2010-01-01", end="2023-05-01")
nasdaq_data = yf.download("^ixic", start="2010-01-01", end="2023-05-01")
ftse100_data = yf.download("^ftse", start="2010-01-01", end="2023-05-01")
sti_data = yf.download("^sti", start="2010-01-01", end="2023-05-01")
kospicomp_data = yf.download("^ks11", start="2010-01-01", end="2023-05-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
msft_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,30.620001,31.100000,30.590000,30.950001,23.572363,38409100
2010-01-05,30.850000,31.100000,30.639999,30.959999,23.579988,49749600
2010-01-06,30.879999,31.080000,30.520000,30.770000,23.435280,58182400
2010-01-07,30.629999,30.700001,30.190001,30.450001,23.191557,50559700
2010-01-08,30.280001,30.879999,30.240000,30.660000,23.351492,51197400
...,...,...,...,...,...,...
2023-04-24,282.089996,284.950012,278.720001,281.769989,281.155365,26611000
2023-04-25,279.510010,281.600006,275.369995,275.420013,274.819244,45772200
2023-04-26,296.700012,299.570007,292.730011,295.369995,294.725708,64599200


In [5]:
def cumulativeReturn(df):
    cr_df = df[['Adj Close']]
    cr = (cr_df.iloc[-1] - cr_df.iloc[0])/(cr_df.iloc[0])
    return cr * 100

In [6]:
def volatility(df):
    vol_df = df['Adj Close']
    return (vol_df.std())*(252**.5)

In [7]:
def sharpeRatio(df):
    sh_df = df[['Adj Close']]
    sh_df['Norm return'] = sh_df['Adj Close'] / sh_df.iloc[0]['Adj Close']
    sh_df['Daily Return'] = sh_df['Adj Close'].pct_change(1)
    sharpe_ratio = (sh_df['Daily Return'].mean() / sh_df['Daily Return'].std()) * (252**.5)
    return sharpe_ratio

In [8]:
def sortinoRatio(df):
    so_df = df[['Adj Close']]
    so_df['Norm return'] = so_df['Adj Close'] / so_df.iloc[0]['Adj Close']
    so_df['Daily Return'] = so_df['Adj Close'].pct_change(1)  
    sortino_ratio = (so_df['Daily Return'].mean() / so_df[so_df['Daily Return'] < 0]['Daily Return'].std()) * (252**.5)
    return sortino_ratio  

In [9]:
def maxDrawDown(df):
    mdd_df = df[['Adj Close']]
    init = mdd_df['Adj Close'][0]
    mdd_df['Cumulative Returns'] = (mdd_df['Adj Close'] - init ) / init 
    mdd_df['Running Max'] = mdd_df['Cumulative Returns'].cummax()
    mdd_df['Drawdowns'] = (mdd_df['Cumulative Returns'] - mdd_df['Running Max'] ) / mdd_df['Running Max']
    return mdd_df['Drawdowns'].min()

In [10]:
equity_list = [msft_data, aapl_data, goog_data, amzn_data, nflx_data]
eqty_names = ["Microsoft", "Apple", "Google", "Amazon", "Netflix"]
index_list = [sp500_data, nasdaq_data, ftse100_data, sti_data, kospicomp_data]
indx_names = ["S&P 500", "NASDAQ", "FTSE 100", "STI Index", "KOSPI Composite Index"]
technique_list = [cumulativeReturn, volatility, sharpeRatio, sortinoRatio, maxDrawDown]
tcnq_name = ["Cumulative Return", "Volatility", "Sharpe Ratio", "Sortino Ratio", "Maximum Drawdown"]

<h1><center> Indices </h1></center>

In [11]:
result = []
for i in range(5):
    op = map(technique_list[i], index_list)
    result.append(list(op))

In [12]:
for i in range(5):
    for j in range(5):
        if i == 0:
            print(f"{tcnq_name[i]} of {indx_names[j]}: {result[i][j].get(key = 'Adj Close')}" )
        else:
            print(f"{tcnq_name[i]} of {indx_names[j]}: {result[i][j]}")
    print("\n")
    

Cumulative Return of S&P 500: 268.0067799721897
Cumulative Return of NASDAQ: 429.65147121906807
Cumulative Return of FTSE 100: 43.09401991049138
Cumulative Return of STI Index: 12.98854587398513
Cumulative Return of KOSPI Composite Index: 47.48369873317165


Volatility of S&P 500: 16008.902642709561
Volatility of NASDAQ: 59598.257487803196
Volatility of FTSE 100: 11466.340957544779
Volatility of STI Index: 3735.494535210287
Volatility of KOSPI Composite Index: 5696.035348467156


Sharpe Ratio of S&P 500: 0.6407634304606422
Sharpe Ratio of NASDAQ: 0.7117707588019104
Sharpe Ratio of FTSE 100: 0.24682030151039425
Sharpe Ratio of STI Index: 0.13622600236634824
Sharpe Ratio of KOSPI Composite Index: 0.2625767647101726


Sortino Ratio of S&P 500: 0.7844535881965914
Sortino Ratio of NASDAQ: 0.8989139850875184
Sortino Ratio of FTSE 100: 0.3187795679294692
Sortino Ratio of STI Index: 0.1807399542175005
Sortino Ratio of KOSPI Composite Index: 0.346799765083835


Maximum Drawdown of S&P 500: -5.4

<h1><center> Equity </h1></center>

In [13]:
eqresult = []
for i in range(5):
    eqop = map(technique_list[i], index_list)
    eqresult.append(list(eqop))

In [14]:
for i in range(5):
    for j in range(5):
        if i == 0:
            print(f"{tcnq_name[i]} of {eqty_names[j]}: {result[i][j].get(key = 'Adj Close')}" )
        else:
            print(f"{tcnq_name[i]} of {eqty_names[j]}: {result[i][j]}")
    print("\n")

Cumulative Return of Microsoft: 268.0067799721897
Cumulative Return of Apple: 429.65147121906807
Cumulative Return of Google: 43.09401991049138
Cumulative Return of Amazon: 12.98854587398513
Cumulative Return of Netflix: 47.48369873317165


Volatility of Microsoft: 16008.902642709561
Volatility of Apple: 59598.257487803196
Volatility of Google: 11466.340957544779
Volatility of Amazon: 3735.494535210287
Volatility of Netflix: 5696.035348467156


Sharpe Ratio of Microsoft: 0.6407634304606422
Sharpe Ratio of Apple: 0.7117707588019104
Sharpe Ratio of Google: 0.24682030151039425
Sharpe Ratio of Amazon: 0.13622600236634824
Sharpe Ratio of Netflix: 0.2625767647101726


Sortino Ratio of Microsoft: 0.7844535881965914
Sortino Ratio of Apple: 0.8989139850875184
Sortino Ratio of Google: 0.3187795679294692
Sortino Ratio of Amazon: 0.1807399542175005
Sortino Ratio of Netflix: 0.346799765083835


Maximum Drawdown of Microsoft: -5.42285633364016
Maximum Drawdown of Apple: -29.85774410774411
Maximum Dr